## CS431/631 Big Data Infrastructure
### Winter 2020 - Assignment 1
---

**Please edit this (text) cell to provide your name and UW student ID number!**
* **Name:** Kamran Karim
* **ID:** 20594936

---
#### Overview
For this assignment, you will be using Python to analyze the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens in the text of Shakespeare's plays.    For this assignment, you will need the same text file (`Shakespeare.txt`) that you used for Assignment 0.   You will also need the Python tokenizer module, `simple_tokenize.py`.

If two events $x$ and $y$ are independent, their PMI will be zero.   A positive PMI indicates that $x$ and $y$ are more likely to co-occur than they would be if they were independent.   Similarly, a negative PMI indicates that $x$ and $y$ are less likely to co-occur.   The PMI of events $x$ and $y$ is given by
\begin{equation*}
PMI(x,y) = \log\frac{p(x,y)}{p(x)p(y)}
\end{equation*}
where $p(x)$ and $p(y)$ are the probabilities of occurrence of events $x$ and $y$, and $p(x,y)$ is the probability of co-occurrence of $x$ and $y$.

For this assignment, the "events" that we are interested in are occurrences of tokens on lines of text in the input file.   For example, one event
might represent the occurence of the token "fire" a line of text, and another might represent the occurrence of the token "peace".   In that case, $p(fire)$ represents the probability that "fire" will occur on a line of text, and $p(fire,peace)$ represents the probability that *both* "fire" and "peace" will occur on the *same* line.   For the purposes of these PMI computations, it does not matter how many times a given token occures on a single line.   Either a line contains a particular token (at least once), or it does not.   For example, consider this line of text:

> three three three, said thrice

For this line, the following token-pair events have occurred:
- (three, said)
- (three, thrice)
- (said, three)
- (said, thrice)
- (thrice, three)
- (thrice, said)

Note that we are not interested in "reflexive" pairs, such as (thrice,thrice).

In addition to the probabilities of events, we will also be interested in the absolute *number* of occurences of particular events, e.g., the number of lines in which "fire" occurs.   We will use $n(x)$ to represent the these numbers.

Your main task for this assignment is to write Python code to analyze the PMI of tokens from Shakespeare's plays.    Based this analysis, we want to be able to answer two types of queries:

* Two-Token Queries: Given a pair of tokens, $x$ and $y$, report the number of lines on which that pair co-occurs ($n(x,y)$) as well as $PMI(x,y)$.
* One-Token Queries: Given a single token, $x$, report the number of lines on which that token occurs ($n(x)$).   In addition, report the five tokens that have the largest PMI with respect to $x$ (and their PMIs).   That is, report the five $y$'s for which $PMI(x,y)$ is largest.

To avoid reporting spurious results for the one-token queries, we are only interested in token pairs that co-occur a sufficient number of times.   Therefore, we will use a *threshold* parameter for one-token queries.   A one-token query should only report pairs of tokens that co-occur at least *threshold* times in the input.   For example, given the threshold 12, a one-token query for "fire" the should report the five tokens that have the largest PMI (with respect to "fire") among all tokens that co-occur with "fire" on at least 12 lines.   If there are fewer than five such tokens, report fewer than five.



---
#### Question 1  (2/10 marks):

Before writing code to handle the PMI queries, start writing some code to answer some simpler questions that give an
idea of how big the PMI analysis problem will be.   The box below contains some starter code that reads in the 'Shakespeare.txt' file and tokenizes it one line at time.   (This is the same code you started with for Assignment 0.)  Extend this code to determine (a) the number of *distinct* tokens that exist in 'Shakespeare.txt', and (b) the number of 
distinct token pairs that exist in 'Shakespeare.txt'.  For the purposes of this question, consider the token pair $x,y$ to be distinct from the pair $y,x$, i.e., count them both.   Ignore token pairs of the form $x,x$.

In [7]:
# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
file = "Shakespeare.txt"
# Now, let's tokenize Shakespeare's plays
def distinct_tokens(filename=file):
    with open(filename) as f:
        di1 = dict()
        di2 = dict()
        for line in f:
            for w in simple_tokenize(line):         #for each word in line\n",
                if w in di1:                         #if word is in dictionary\n",
                    di1[w] +=1                       #increase the value by one  \n",
                else:
                    di1[w] =1
                for w2 in simple_tokenize(line):
                    if w != w2:
                        if (w, w2) in di2:
                            di2[(w, w2)] +=1                       #increase the value by one  \n",
                        else:
                            di2[(w, w2)] =1
    uni_tokens =  len(di1)                                 #extracts the value of unique tokens,
    uni_token_pair = len(di2)                              #extracts the value of unique token pairs

    return  (uni_tokens, uni_token_pair)        


# extend this code to answer Question 1.
# when your code is executed, it should print the number of distinct tokens and the number of distinct token pairs

In [8]:

distinct_tokens() #prints (unique tokens, unique token pairs)

(25975, 1969760)

---

#### Question 2 (6/10 marks):
Next, write Python code to answer the one-token and two-token queries described above, for 'Shakespeare.txt'.   The code cell below contains some starter code that implements a simple text-based query interface.  It allows a user to ask a series of one-token or two-token queries.   Try running the starter code to get a sense of how the interface behaves.    

Your task is to write code to read and tokenize 'Shakespeare.txt', record information that will allow both types of PMI queries to be answered, and then answer queries that are posed through the query interface.  Make sure that your code is well commented, so that it will be clear to the markers.

If you cannot get answers to both types of queries working, try to get at least one type working, for partial credit.


# Note: file name will be required to test this code: file name is "Shakespeare.txt"

In [11]:

# this imports the SimpleTokenize function from the simple_tokenize.py file that you uploaded
from simple_tokenize import simple_tokenize
# the log function for computing PMI
# for the sake of consistency across solutions, please use log base 10
from math import log

def n_pmi(token, file, tr):                        #function for giving out the pmi inputs when there is one token
    with open(file) as f:
        di = dict()                                #uses dictionary
        n=0                                        #keeps tract of total number 
        m = 0                                      #keeps tract of the token entries in terms of lines
        for line in f:
            n+=1
            t=list()                               #checks if the word has already be compared, if it is repeated in one line
            if token in simple_tokenize(line):
                m+=1                                      
                for w in simple_tokenize(line):    
                    if w != token:                 #if the word is not same as token
                        if w in t:                 #and word for that line has not already been counted
                            continue
                        elif w not in t:
                            
                            if (token,w) in di:
                                di[(token,w)] +=1 #add to dictionary
                            elif (token,w) not in di:
                                di[(token,w)] = 1
                        t.append(w)               #keep track of what word is seen in a particular line
            #else if line not in f:
            #    continue
        d2 = di.items()                                 #stores the dictionary as a array of tuples
        d3 = sorted(d2, key = lambda t : t[1], reverse = True) #sorts the array in descending order by value 
        d = d3[0:5]
        d4 = list()
        for i in d:
            if i[1] >tr:
                d4.append(i)
                
       
        
                
        #ind = (d > tr).nonzero()[0]
        
    return (d4, n, m)



def count_ln(token, file):            #helper function
    with open(file) as f:
        m = 0
        for line in f:
            if token in simple_tokenize(line):
                    m+=1    
    return m


def n_pmi_pair(token1, token2, file, tr):                       #npi inputs for pairs
    with open(file) as f:
        di1 = dict()                                            #different variables that are later used
        tk1 = 0
        tk2 = 0
        n= 0
        c= 0
        for line in f:
            n+=1
            a= 0
            b= 0
            lk = simple_tokenize(line)
            if (token1 in lk):
                tk1+= 1
                a=1
            if (token2 in lk):
                tk2+= 1
                b=1
            if (a==1 & b==1):
                c += 1
    return (tk1, tk2, c, n)

while True:
    q = input("Input 1 or 2 space-separated tokens (return to quit): ")
    file = input("Please input the name of the file you want to do the PMI operations in (return to quit): ")
    if len(q) == 0:
        break
    q_tokens = simple_tokenize(q)
    if len(q_tokens) == 1:
        threshold = 0
        while threshold <= 0:
            try:
                threshold = int(input("Input a positive integer frequency threshold: "))
            except ValueError:
                print("Threshold must be a positive integer!")
                continue
            
                
                
        l_pmi = n_pmi(q, file, threshold)[0]#[0]#[0][0][0]
        tot   = n_pmi(q, file, threshold)[1]
        tok1  = n_pmi(q, file, threshold)[2]        
        
        # Put code here to answer a One-Token Query with token q_tokens[0] and the specified threshold,
        # and output the result.
        # The print() statements below exist to show you the desired output format.
        # Replace them with your own output code, which should produce results in a similar format.
        if [] not in n_pmi(q, file, threshold):
            print(("  n("+ str(l_pmi[0][0][0])+ ") = " + str(n_pmi(q, file, threshold)[2])).format(q_tokens[0]))
            print(("  high PMI tokens with respect to" + str(q) + "(threshold: "+ str(threshold)+"):" ) .format(q_tokens[0],threshold))
            
            #Note the case when the common lines for two tokens are 0 is not taken 
            #into account as the minimum value of threshold is 0, hence common 
            #values >= 1
    
            for i in range(len(n_pmi(q, file, threshold)[0])):
                print(("    n(" + str(l_pmi[i][0])+ ") ="+ 
                       str(l_pmi[i][1]) + ",   PMI = " +
                       str( log(((l_pmi[i][1]/tot)/((count_ln(l_pmi[i][0][0], file)/tot) *
                                  (count_ln(l_pmi[i][0][1], file)/tot) ) ),10 ))
                       ))    
        else:
            print("The word does not lie in this file")
        
        
        
        #print(("    " + l_pmi[0][0][0]+" ="+ XXX + ",   PMI" +l_pmi[0][0][0]+" =" + Y.YYY ).format(q_tokens[0]))
            #print(("    " + l_pmi[0][0][0]+" ="+ XXX + ",   PMI" +l_pmi[0][0][0]+" =" + Y.YYY ).format(q_tokens[0]))
            #print(("    " + l_pmi[0][0][0]+" ="+ XXX + ",   PMI" +l_pmi[0][0][0]+" =" + Y.YYY ).format(q_tokens[0]))
            #print(("    " + l_pmi[0][0][0]+" ="+ XXX + ",   PMI" +l_pmi[0][0][0]+" =" + Y.YYY ).format(q_tokens[0]))
        # in the above, all XXX values should be at least as large as the threshold





    elif len(q_tokens) == 2:
        threshold = 0
        # Put code here to answer a Two-Token Query with tokens q_tokens[0] and q_tokens[1]
        # As was the case for the One-Token query, the print statements below show the desired output format
        # Replace them with your own output code
        k = n_pmi_pair(q_tokens[0], q_tokens[1], file, threshold)
        print("    n(" +str(q_tokens[0]) + ", "+ str(q_tokens[1]) + ") = " + str(k[2]) )#.format(q_tokens[0],q_tokens[1]))
        print(("  PMI(" +str(q_tokens[0]) + "," +str(q_tokens[1]) + ") = " + 
               str( log(((k[2]/k[3])/((k[0]/k[3])*(k[1]/k[3]))), 10))   
               ))#.format(q_tokens[0],q_tokens[1]))
    else:
        print("Input must consist of 1 or 2 space-separated tokens!")

Input 1 or 2 space-separated tokens (return to quit): sfer
Please input the name of the file you want to do the PMI operations in (return to quit): Shakespeare.txt
Input a positive integer frequency threshold: 4
The word does not lie in this file
Input 1 or 2 space-separated tokens (return to quit): 
Please input the name of the file you want to do the PMI operations in (return to quit): 


---

#### Question 3 (2/10 marks):

Suppose that you try to run your PMI analysis on larger files:  say, 10 times, or 100 times, or 1000 times larger than 'Shakespeare.txt'.    As the input file grows larger, what will happen to the execution of your program?   Will it get slower?   How much slower?   Will it eventually fail to run?   If so, why?

In the cell below, briefly (one or two paragraphs), discuss what will happen if the input to your analysis grows.  We're not looking for an exact or empirical analysis of the behaviour of your program as a function of input size.  Rather, we are looking for a discussion of trends and limits.

#### Answer to Question 3:
The program will definitely go slower as we increase the size of the file. Since the way our code is writen, most calculations require going through each word. So first we check if the token is in the tokenised line and then we check all other words. Our run time will be directly propotional to the number of words in the document or O(n). As the program gets very big, then it will take much more time.

Now, since we are using dictionary, which is stored in memory, now if the program gets very big, and we keep on adding new vocabulary, there may come a time when the dictionary goes out of memory capacity. But, note that there is marginal addition to the vocabulary, no matter how big the file is, hence, this may not happen. So if it did not happen with lets say 5 GB of text in the same language and context, the out of memory is very less likely to happen with similar kind of data with 1 TB of text as the dictionary values only change, not many new keys would be added. but the program will indeed get very slow.


---
Don't forget to save your workbook!   (It's a good idea to do this regularly, while you are working.)   When you are finished and you are ready to submit your assignment, download your notebook file (.ipynb) from the hub to your machine, and then follow the submission instructions in the assignment.